In [1]:
%%capture

## Import Libraries
import numpy as np # linear algebra
from numpy.random import seed 
import math 

import pandas as pd # data processing 
pd.options.display.max_rows = 1000
pd.options.display.max_seq_items = 2000
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

import datetime as dt
from datatable import dt, f

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.dates as mpl_dates
plt.rcParams.update({'font.size': 14})
import seaborn as sns
plt.style.use('seaborn')
sns.set_style('whitegrid')


from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

import warnings # Supress warnings 
warnings.filterwarnings('ignore')

import statsmodels as sm

import joblib

# Fix seed for reproducible results
SEED = 42
np.random.seed(SEED)

## Import Data

In [2]:
### Used method from Kaggle to reduce memory usage. 
## Don't know original source
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [3]:
%%time

## Import data
data_folder = "../input/ubiquant-market-prediction/"
data_folder_parquet = "../input/ubiquant-parquet/"

def read_datasets(path):
    print('Loading Example Sample Submission file...')
    example_submission = pd.read_parquet(path + "example_sample_submission.parquet")
    display(example_submission.head())
    print('Loading Example Test file...')
    test = pd.read_parquet(path + "example_test.parquet")
    display(test.head())
    print('Loading Train file...')
    train = pd.read_parquet(path + "train_low_mem.parquet").drop(columns=['row_id'])
    train = reduce_mem_usage(train)
    display(train.head())
    
    return train, test, example_submission

train, test, example_submission = read_datasets(data_folder_parquet)

Loading Example Sample Submission file...


,row_id,time_id,target
0,1220_1,1220,0
1,1220_2,1220,0
2,1221_0,1221,0
3,1221_1,1221,0
4,1221_2,1221,0


Loading Example Test file...


,row_id,time_id,investment_id,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,f_31,f_32,f_33,f_34,f_35,f_36,f_37,f_38,f_39,f_40,f_41,f_42,f_43,f_44,f_45,f_46,f_47,f_48,f_49,f_50,f_51,f_52,f_53,f_54,f_55,f_56,f_57,f_58,f_59,f_60,f_61,f_62,f_63,f_64,f_65,f_66,f_67,f_68,f_69,f_70,f_71,f_72,f_73,f_74,f_75,f_76,f_77,f_78,f_79,f_80,f_81,f_82,f_83,f_84,f_85,f_86,f_87,f_88,f_89,f_90,f_91,f_92,f_93,f_94,f_95,f_96,f_97,f_98,f_99,f_100,f_101,f_102,f_103,f_104,f_105,f_106,f_107,f_108,f_109,f_110,f_111,f_112,f_113,f_114,f_115,f_116,f_117,f_118,f_119,f_120,f_121,f_122,f_123,f_124,f_125,f_126,f_127,f_128,f_129,f_130,f_131,f_132,f_133,f_134,f_135,f_136,f_137,f_138,f_139,f_140,f_141,f_142,f_143,f_144,f_145,f_146,f_147,f_148,f_149,f_150,f_151,f_152,f_153,f_154,f_155,f_156,f_157,f_158,f_159,f_160,f_161,f_162,f_163,f_164,f_165,f_166,f_167,f_168,f_169,f_170,f_171,f_172,f_173,f_174,f_175,f_176,f_177,f_178,f_179,f_180,f_181,f_182,f_183,f_184,f_185,f_186,f_187,f_188,f_189,f_190,f_191,f_192,f_193,f_194,f_195,f_196,f_197,f_198,f_199,f_200,f_201,f_202,f_203,f_204,f_205,f_206,f_207,f_208,f_209,f_210,f_211,f_212,f_213,f_214,f_215,f_216,f_217,f_218,f_219,f_220,f_221,f_222,f_223,f_224,f_225,f_226,f_227,f_228,f_229,f_230,f_231,f_232,f_233,f_234,f_235,f_236,f_237,f_238,f_239,f_240,f_241,f_242,f_243,f_244,f_245,f_246,f_247,f_248,f_249,f_250,f_251,f_252,f_253,f_254,f_255,f_256,f_257,f_258,f_259,f_260,f_261,f_262,f_263,f_264,f_265,f_266,f_267,f_268,f_269,f_270,f_271,f_272,f_273,f_274,f_275,f_276,f_277,f_278,f_279,f_280,f_281,f_282,f_283,f_284,f_285,f_286,f_287,f_288,f_289,f_290,f_291,f_292,f_293,f_294,f_295,f_296,f_297,f_298,f_299
0,1220_1,1220,1,0.874944,0.457875,0.962572,0.372061,0.004509,0.971304,0.097477,0.246080,0.481052,0.951130,0.968358,0.699345,0.739139,0.066587,0.808991,0.291094,0.410428,0.415464,0.475559,0.463030,0.162172,0.764135,0.988812,0.620314,0.915356,0.810174,0.793990,0.276981,0.777230,0.806242,0.094767,0.254653,0.065229,0.172443,0.429970,0.375311,0.235271,0.979103,0.739211,0.946839,0.328391,0.552147,0.291854,0.475741,0.786556,0.898680,0.115453,0.800245,0.804779,0.185797,0.230276,0.893254,0.318648,0.213987,0.186194,0.681819,0.393373,0.480479,0.473462,0.605556,0.723612,0.774659,0.326204,0.988071,0.697630,0.561397,0.748171,0.087270,0.317911,0.551137,0.733891,0.685182,0.721296,0.840730,0.990324,0.036338,0.453641,0.663457,0.109746,0.978097,0.785883,0.085453,0.101547,0.088554,0.773017,0.671386,0.260844,0.138067,0.209517,0.432560,0.543269,0.710578,0.513228,0.616471,0.400029,0.577039,0.800661,0.340733,0.522114,0.109640,0.915578,0.654280,0.103689,0.485371,0.531522,0.903273,0.265315,0.777290,0.299357,0.813668,0.616994,0.852711,0.652874,0.670269,0.599262,0.955137,0.576315,0.647672,0.965780,0.721828,0.643987,0.688608,0.148123,0.654623,0.400646,0.550776,0.927801,0.351822,0.550095,0.548258,0.565763,0.886405,0.445483,0.635064,0.228078,0.999351,0.132959,0.719205,0.079829,0.896854,0.170177,0.462065,0.069810,0.612618,0.501901,0.275765,0.452534,0.830298,0.097434,0.448382,0.138786,0.306385,0.176685,0.888951,0.296462,0.691869,0.123614,0.048984,0.965639,0.619477,0.144715,0.064928,0.641233,0.089973,0.385225,0.071957,0.437211,0.368989,0.688976,0.150254,0.993027,0.099046,0.892578,0.030713,0.241537,0.794321,0.587304,0.289077,0.154698,0.786947,0.821293,0.122905,0.163158,0.304153,0.246596,0.070908,0.095635,0.554278,0.319731,0.474685,0.850515,0.151222,0.841399,0.009890,0.074472,0.966512,0.273875,0.687070,0.035826,0.525475,0.321969,0.594598,0.079995,0.575756,0.207780,0.455963,0.933001,0.145935,0.660078,0.173596,0.654387,0.470070,0.744501,0.721583,0.336405,0.632004,0.901299,0.448609,0.686336,0.276582,0.495469,0.397506,0.671769,0.385539,0.453976,0.037179,0.758213,0.461456,0.774596,0.391214,0.803420,0.562853,0.162382,0.506769,0.794830,0.268479,0.748373,0.044955,0.867946,0.813450,0.816583,0.842692,0.204182,0.790365,0.051641,0.718111,0.299606,0.082191,0.851497,0.690467,0.774980,0.306

Loading Train file...
Memory usage of dataframe is 3619.03 MB
Memory usage after optimization is: 1815.50 MB
Decreased by 49.8%


,time_id,investment_id,target,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,f_31,f_32,f_33,f_34,f_35,f_36,f_37,f_38,f_39,f_40,f_41,f_42,f_43,f_44,f_45,f_46,f_47,f_48,f_49,f_50,f_51,f_52,f_53,f_54,f_55,f_56,f_57,f_58,f_59,f_60,f_61,f_62,f_63,f_64,f_65,f_66,f_67,f_68,f_69,f_70,f_71,f_72,f_73,f_74,f_75,f_76,f_77,f_78,f_79,f_80,f_81,f_82,f_83,f_84,f_85,f_86,f_87,f_88,f_89,f_90,f_91,f_92,f_93,f_94,f_95,f_96,f_97,f_98,f_99,f_100,f_101,f_102,f_103,f_104,f_105,f_106,f_107,f_108,f_109,f_110,f_111,f_112,f_113,f_114,f_115,f_116,f_117,f_118,f_119,f_120,f_121,f_122,f_123,f_124,f_125,f_126,f_127,f_128,f_129,f_130,f_131,f_132,f_133,f_134,f_135,f_136,f_137,f_138,f_139,f_140,f_141,f_142,f_143,f_144,f_145,f_146,f_147,f_148,f_149,f_150,f_151,f_152,f_153,f_154,f_155,f_156,f_157,f_158,f_159,f_160,f_161,f_162,f_163,f_164,f_165,f_166,f_167,f_168,f_169,f_170,f_171,f_172,f_173,f_174,f_175,f_176,f_177,f_178,f_179,f_180,f_181,f_182,f_183,f_184,f_185,f_186,f_187,f_188,f_189,f_190,f_191,f_192,f_193,f_194,f_195,f_196,f_197,f_198,f_199,f_200,f_201,f_202,f_203,f_204,f_205,f_206,f_207,f_208,f_209,f_210,f_211,f_212,f_213,f_214,f_215,f_216,f_217,f_218,f_219,f_220,f_221,f_222,f_223,f_224,f_225,f_226,f_227,f_228,f_229,f_230,f_231,f_232,f_233,f_234,f_235,f_236,f_237,f_238,f_239,f_240,f_241,f_242,f_243,f_244,f_245,f_246,f_247,f_248,f_249,f_250,f_251,f_252,f_253,f_254,f_255,f_256,f_257,f_258,f_259,f_260,f_261,f_262,f_263,f_264,f_265,f_266,f_267,f_268,f_269,f_270,f_271,f_272,f_273,f_274,f_275,f_276,f_277,f_278,f_279,f_280,f_281,f_282,f_283,f_284,f_285,f_286,f_287,f_288,f_289,f_290,f_291,f_292,f_293,f_294,f_295,f_296,f_297,f_298,f_299
0,0.0,1.0,-0.300781,0.932617,0.113708,-0.402100,0.378418,-0.203979,-0.413574,0.965820,1.230469,0.114807,-2.013672,0.004936,0.284180,0.501953,-0.287842,-1.168945,-0.267334,-0.574219,-0.771973,1.012695,-1.230469,1.786133,-2.089844,0.325684,-0.877930,1.048828,0.131714,-0.349609,-1.813477,0.099243,-0.240967,1.604492,0.003637,-0.901855,0.221558,0.609863,-0.738770,2.097656,-0.914062,-0.293945,-0.037994,0.685547,0.0,0.414795,-0.469482,-1.085938,-0.106445,0.059448,0.483154,1.189453,0.506836,0.754395,0.317627,-0.183228,5.164062,0.677246,-0.440674,0.631348,-1.845703,0.227783,0.841309,0.667480,-0.854004,-0.17041,-0.11969,-0.504883,0.662598,1.933594,-0.812500,0.072815,-0.778809,-0.553711,-0.032776,-0.619141,-1.424805,0.148315,0.257812,0.735352,0.563965,-0.272217,1.190430,-0.101379,1.333984,0.191284,0.630859,1.010742,0.161865,0.609375,-0.219482,-0.013069,1.505859,-1.319336,0.448242,-0.635254,-0.075378,1.033203,-0.664062,0.139893,0.186279,0.593262,-0.301514,0.666504,0.713379,1.020508,0.879883,-1.096680,-0.211060,-0.117493,-0.778809,0.065979,0.229736,0.435303,1.233398,-0.865723,0.062347,0.382324,-1.315430,0.786621,0.770508,-0.871094,0.583496,0.115601,0.209595,0.325439,0.469238,0.0,-0.731445,-0.249512,-0.050568,-1.232422,-0.790527,0.493408,0.357910,-0.381592,0.176392,0.748047,-0.741699,0.089661,1.080078,1.137695,1.199219,0.030609,-0.356201,0.247803,1.395508,0.961914,0.078125,-1.163086,0.707520,0.256104,-0.026306,0.329102,0.196045,0.653320,0.069214,0.907715,-0.350586,-1.170898,0.659180,0.937500,0.482910,-0.094910,0.317383,0.126831,0.892090,-1.199219,-0.315430,0.704102,-0.017700,-0.513184,-0.362549,1.0,2.265625,0.360107,0.054474,1.639648,-1.0,0.876953,0.368164,0.991211,-1.224609,-1.208984,-0.879883,1.0,0.035156,0.693359,0.307129,0.143799,0.728027,1.220703,-0.939941,-0.106934,0.000000,-0.569336,0.187866,-0.386475,-0.452393,0.269775,-0.565430,-0.262207,-1.473633,0.0,0.428223,1.740234,0.941406,-0.729980,-0.533203,-0.983398,-0.043152,-0.225952,-0.008301,-0.447998,0.551270,-0.153076,-2.292969,0.186646,-0.443115,0.121216,0.787598,1.996094,-1.286133,-0.157227,1.019531,0.693359,0.797363,-0.192627,0.222778,0.921387,-0.171387,0.021652,1.153320,0.689941,3.251953,0.992188,-0.779785,-0.546387,-0.379150,1.896484,-0.732422,-0.113770,1.127930,-0.007927,0.122314,0.444580,0.4

CPU times: user 1min 28s, sys: 2min 1s, total: 3min 29s
Wall time: 3min 50s


## Dealing with Multicollinearity
### Remove highly correlated features using VIF

In [4]:
### Important! ###
## This method fails and returns a 'Out of memory error' in Kaggle kernel. 
## I calculated the VIF seperately in a Google Colab notebook that did finish
## 44 columns were identified to be dropped from the calculate_vif() method
## ['f_148', 'f_95', 'f_277', 'f_61', 'f_173', 'f_72', 'f_206', 'f_254', 'f_97', 'f_273', 'f_262', 'f_180', 'f_51', 'f_54', 'f_214', 'f_203', 'f_106', 'f_134', 'f_228', 'f_28', 'f_255', 'f_247', 'f_280', 'f_136', 'f_164', 'f_4', 'f_205', 'f_186', 'f_193', 'f_15', 'f_212', 'f_161', 'f_157', 'f_146', 'f_274', 'f_240', 'f_211', 'f_3', 'f_158', 'f_41', 'f_288', 'f_207', 'f_245', 'f_226']

# Get a list of all the features first and the target
features = [f"f_{i}" for i in range(300)]
target = 'target'

# Generate a df with only features
df_features = train.loc[:, features]
display(df_features.shape)

### Important! ###
## This method fails and returns a 'Out of memory error in '
# Calculate VIF
def calculate_vif(df, features): 
    vif, tolerance = {}, {}

    for feature in features:
        X = [f for f in features if f != feature]
        X, y = df[X], df[feature]
        r2 = LinearRegression().fit(X, y).score(X, y)
        tolerance[feature] = 1 - r2
        vif[feature] = 1/(tolerance[feature])
    return pd.DataFrame({'VIF': vif, 'Tolerance': tolerance})

# dfx = calculate_vif(df_features, features)

# drop_columns = set()
# mc_threshold = 5 
# for feature in features:
#     if dfx.loc[feature].VIF > mc_threshold:
#         drop_columns.add(feature)

drop_columns = ['f_148', 'f_95', 'f_277', 'f_61', 'f_173', 'f_72', 'f_206', 'f_254', 'f_97', 'f_273', 'f_262', 'f_180', 'f_51', 'f_54', 'f_214', 'f_203', 'f_106', 'f_134', 'f_228', 'f_28', 'f_255', 'f_247', 'f_280', 'f_136', 'f_164', 'f_4', 'f_205', 'f_186', 'f_193', 'f_15', 'f_212', 'f_161', 'f_157', 'f_146', 'f_274', 'f_240', 'f_211', 'f_3', 'f_158', 'f_41', 'f_288', 'f_207', 'f_245', 'f_226']
df_features = df_features.drop(columns=drop_columns)
print(f'Columns drop size : {len(drop_columns)}')
display(df_features.head())

(3141410, 300)

Columns drop size : 44


,f_0,f_1,f_2,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_29,f_30,f_31,f_32,f_33,f_34,f_35,f_36,f_37,f_38,f_39,f_40,f_42,f_43,f_44,f_45,f_46,f_47,f_48,f_49,f_50,f_52,f_53,f_55,f_56,f_57,f_58,f_59,f_60,f_62,f_63,f_64,f_65,f_66,f_67,f_68,f_69,f_70,f_71,f_73,f_74,f_75,f_76,f_77,f_78,f_79,f_80,f_81,f_82,f_83,f_84,f_85,f_86,f_87,f_88,f_89,f_90,f_91,f_92,f_93,f_94,f_96,f_98,f_99,f_100,f_101,f_102,f_103,f_104,f_105,f_107,f_108,f_109,f_110,f_111,f_112,f_113,f_114,f_115,f_116,f_117,f_118,f_119,f_120,f_121,f_122,f_123,f_124,f_125,f_126,f_127,f_128,f_129,f_130,f_131,f_132,f_133,f_135,f_137,f_138,f_139,f_140,f_141,f_142,f_143,f_144,f_145,f_147,f_149,f_150,f_151,f_152,f_153,f_154,f_155,f_156,f_159,f_160,f_162,f_163,f_165,f_166,f_167,f_168,f_169,f_170,f_171,f_172,f_174,f_175,f_176,f_177,f_178,f_179,f_181,f_182,f_183,f_184,f_185,f_187,f_188,f_189,f_190,f_191,f_192,f_194,f_195,f_196,f_197,f_198,f_199,f_200,f_201,f_202,f_204,f_208,f_209,f_210,f_213,f_215,f_216,f_217,f_218,f_219,f_220,f_221,f_222,f_223,f_224,f_225,f_227,f_229,f_230,f_231,f_232,f_233,f_234,f_235,f_236,f_237,f_238,f_239,f_241,f_242,f_243,f_244,f_246,f_248,f_249,f_250,f_251,f_252,f_253,f_256,f_257,f_258,f_259,f_260,f_261,f_263,f_264,f_265,f_266,f_267,f_268,f_269,f_270,f_271,f_272,f_275,f_276,f_278,f_279,f_281,f_282,f_283,f_284,f_285,f_286,f_287,f_289,f_290,f_291,f_292,f_293,f_294,f_295,f_296,f_297,f_298,f_299
0,0.932617,0.113708,-0.402100,-0.413574,0.965820,1.230469,0.114807,-2.013672,0.004936,0.284180,0.501953,-0.287842,-1.168945,-0.574219,-0.771973,1.012695,-1.230469,1.786133,-2.089844,0.325684,-0.877930,1.048828,0.131714,-0.349609,-1.813477,-0.240967,1.604492,0.003637,-0.901855,0.221558,0.609863,-0.738770,2.097656,-0.914062,-0.293945,-0.037994,0.685547,0.414795,-0.469482,-1.085938,-0.106445,0.059448,0.483154,1.189453,0.506836,0.754395,-0.183228,5.164062,-0.440674,0.631348,-1.845703,0.227783,0.841309,0.667480,-0.17041,-0.11969,-0.504883,0.662598,1.933594,-0.812500,0.072815,-0.778809,-0.553711,-0.032776,-1.424805,0.148315,0.257812,0.735352,0.563965,-0.272217,1.190430,-0.101379,1.333984,0.191284,0.630859,1.010742,0.161865,0.609375,-0.219482,-0.013069,1.505859,-1.319336,0.448242,-0.635254,-0.075378,1.033203,0.139893,0.593262,-0.301514,0.666504,0.713379,1.020508,0.879883,-1.096680,-0.211060,-0.778809,0.065979,0.229736,0.435303,1.233398,-0.865723,0.062347,0.382324,-1.315430,0.786621,0.770508,-0.871094,0.583496,0.115601,0.209595,0.325439,0.469238,0.0,-0.731445,-0.249512,-0.050568,-1.232422,-0.790527,0.493408,0.357910,-0.381592,0.176392,-0.741699,1.080078,1.137695,1.199219,0.030609,-0.356201,0.247803,1.395508,0.961914,0.078125,0.707520,-0.026306,0.329102,0.196045,0.653320,0.069214,0.907715,-0.350586,-1.170898,0.482910,-0.094910,0.126831,0.892090,-0.315430,0.704102,-0.017700,-0.513184,-0.362549,1.0,2.265625,0.360107,1.639648,-1.0,0.876953,0.368164,0.991211,-1.224609,-0.879883,1.0,0.035156,0.693359,0.307129,0.728027,1.220703,-0.939941,-0.106934,0.000000,-0.569336,-0.386475,-0.452393,0.269775,-0.565430,-0.262207,-1.473633,0.0,0.428223,1.740234,-0.729980,-0.225952,-0.008301,-0.447998,-2.292969,-0.443115,0.121216,0.787598,1.996094,-1.286133,-0.157227,1.019531,0.693359,0.797363,-0.192627,0.222778,-0.171387,1.153320,0.689941,3.251953,0.992188,-0.779785,-0.546387,-0.379150,1.896484,-0.732422,-0.113770,1.127930,0.122314,0.444580,0.485840,-2.302734,0.000000,-0.893555,0.055939,0.201416,-0.338135,1.278320,1.367188,0.145264,0.346924,-0.824219,-1.306641,-1.158203,0.709473,-1.020508,-1.291016,0.038666,0.187134,-0.680176,0.900391,-0.924805,-1.057617,-0.167114,0.0,-0.237915,-0.742188,0.992676,0.961426,-0.006260,0.473633,0.040131,0.453613,-1.597656,0.301758,0.157471,1.505859,0.365967,-1.095703,0.200073,0.819336,0.941406,-0.086792,-1.086914,-1.044922,-0.287598,0.321533
1,0.811035,-0.514160,0.742188,1.771484,1.427734,1.133789,0.114807,-0.219238,-0.351807,0.846680,0.440186,0.499756,0.893066,-0.681641,1.253906,-1.027344,-1.690430,0.011154,0.87548

In [5]:
## Include back target to have a df with features for modeling and the target
columns = list(df_features.columns)
columns.insert(0, target)
df_features_target = train.loc[:, columns]
display(df_features_target.head())

,target,f_0,f_1,f_2,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_29,f_30,f_31,f_32,f_33,f_34,f_35,f_36,f_37,f_38,f_39,f_40,f_42,f_43,f_44,f_45,f_46,f_47,f_48,f_49,f_50,f_52,f_53,f_55,f_56,f_57,f_58,f_59,f_60,f_62,f_63,f_64,f_65,f_66,f_67,f_68,f_69,f_70,f_71,f_73,f_74,f_75,f_76,f_77,f_78,f_79,f_80,f_81,f_82,f_83,f_84,f_85,f_86,f_87,f_88,f_89,f_90,f_91,f_92,f_93,f_94,f_96,f_98,f_99,f_100,f_101,f_102,f_103,f_104,f_105,f_107,f_108,f_109,f_110,f_111,f_112,f_113,f_114,f_115,f_116,f_117,f_118,f_119,f_120,f_121,f_122,f_123,f_124,f_125,f_126,f_127,f_128,f_129,f_130,f_131,f_132,f_133,f_135,f_137,f_138,f_139,f_140,f_141,f_142,f_143,f_144,f_145,f_147,f_149,f_150,f_151,f_152,f_153,f_154,f_155,f_156,f_159,f_160,f_162,f_163,f_165,f_166,f_167,f_168,f_169,f_170,f_171,f_172,f_174,f_175,f_176,f_177,f_178,f_179,f_181,f_182,f_183,f_184,f_185,f_187,f_188,f_189,f_190,f_191,f_192,f_194,f_195,f_196,f_197,f_198,f_199,f_200,f_201,f_202,f_204,f_208,f_209,f_210,f_213,f_215,f_216,f_217,f_218,f_219,f_220,f_221,f_222,f_223,f_224,f_225,f_227,f_229,f_230,f_231,f_232,f_233,f_234,f_235,f_236,f_237,f_238,f_239,f_241,f_242,f_243,f_244,f_246,f_248,f_249,f_250,f_251,f_252,f_253,f_256,f_257,f_258,f_259,f_260,f_261,f_263,f_264,f_265,f_266,f_267,f_268,f_269,f_270,f_271,f_272,f_275,f_276,f_278,f_279,f_281,f_282,f_283,f_284,f_285,f_286,f_287,f_289,f_290,f_291,f_292,f_293,f_294,f_295,f_296,f_297,f_298,f_299
0,-0.300781,0.932617,0.113708,-0.402100,-0.413574,0.965820,1.230469,0.114807,-2.013672,0.004936,0.284180,0.501953,-0.287842,-1.168945,-0.574219,-0.771973,1.012695,-1.230469,1.786133,-2.089844,0.325684,-0.877930,1.048828,0.131714,-0.349609,-1.813477,-0.240967,1.604492,0.003637,-0.901855,0.221558,0.609863,-0.738770,2.097656,-0.914062,-0.293945,-0.037994,0.685547,0.414795,-0.469482,-1.085938,-0.106445,0.059448,0.483154,1.189453,0.506836,0.754395,-0.183228,5.164062,-0.440674,0.631348,-1.845703,0.227783,0.841309,0.667480,-0.17041,-0.11969,-0.504883,0.662598,1.933594,-0.812500,0.072815,-0.778809,-0.553711,-0.032776,-1.424805,0.148315,0.257812,0.735352,0.563965,-0.272217,1.190430,-0.101379,1.333984,0.191284,0.630859,1.010742,0.161865,0.609375,-0.219482,-0.013069,1.505859,-1.319336,0.448242,-0.635254,-0.075378,1.033203,0.139893,0.593262,-0.301514,0.666504,0.713379,1.020508,0.879883,-1.096680,-0.211060,-0.778809,0.065979,0.229736,0.435303,1.233398,-0.865723,0.062347,0.382324,-1.315430,0.786621,0.770508,-0.871094,0.583496,0.115601,0.209595,0.325439,0.469238,0.0,-0.731445,-0.249512,-0.050568,-1.232422,-0.790527,0.493408,0.357910,-0.381592,0.176392,-0.741699,1.080078,1.137695,1.199219,0.030609,-0.356201,0.247803,1.395508,0.961914,0.078125,0.707520,-0.026306,0.329102,0.196045,0.653320,0.069214,0.907715,-0.350586,-1.170898,0.482910,-0.094910,0.126831,0.892090,-0.315430,0.704102,-0.017700,-0.513184,-0.362549,1.0,2.265625,0.360107,1.639648,-1.0,0.876953,0.368164,0.991211,-1.224609,-0.879883,1.0,0.035156,0.693359,0.307129,0.728027,1.220703,-0.939941,-0.106934,0.000000,-0.569336,-0.386475,-0.452393,0.269775,-0.565430,-0.262207,-1.473633,0.0,0.428223,1.740234,-0.729980,-0.225952,-0.008301,-0.447998,-2.292969,-0.443115,0.121216,0.787598,1.996094,-1.286133,-0.157227,1.019531,0.693359,0.797363,-0.192627,0.222778,-0.171387,1.153320,0.689941,3.251953,0.992188,-0.779785,-0.546387,-0.379150,1.896484,-0.732422,-0.113770,1.127930,0.122314,0.444580,0.485840,-2.302734,0.000000,-0.893555,0.055939,0.201416,-0.338135,1.278320,1.367188,0.145264,0.346924,-0.824219,-1.306641,-1.158203,0.709473,-1.020508,-1.291016,0.038666,0.187134,-0.680176,0.900391,-0.924805,-1.057617,-0.167114,0.0,-0.237915,-0.742188,0.992676,0.961426,-0.006260,0.473633,0.040131,0.453613,-1.597656,0.301758,0.157471,1.505859,0.365967,-1.095703,0.200073,0.819336,0.941406,-0.086792,-1.086914,-1.044922,-0.287598,0.321533
1,-0.231079,0.811035,-0.514160,0.742188,1.771484,1.427734,1.133789,0.114807,-0.219238,-0.351807,0.846680,0.440186,0.499756,0.893066,-0.681641,1.253906,-1.027344

## Modeling

In [6]:
%%time 
from sklearn.model_selection import train_test_split

def prep_data_for_modeling(df):
    X = df.drop(columns='target')
    y = df['target']
    print(f"X shape: {X.shape}")
    print(f"y shape: {y.shape}")
    print('\n')

    x_train,x_test,y_train,y_test = train_test_split(X, y, test_size=0.3, random_state=1111)
    print(f"X_train shape: {x_train.shape}")
    print(f"y_train shape: {y_train.shape}")
    print(f"X_test shape: {x_test.shape}")
    print(f"y_test shape: {y_test.shape}")
    
    return x_train, y_train, x_test, y_test

x_train, y_train, x_test, y_test = prep_data_for_modeling(df_features_target)

X shape: (3141410, 256)
y shape: (3141410,)


X_train shape: (2198987, 256)
y_train shape: (2198987,)
X_test shape: (942423, 256)
y_test shape: (942423,)
CPU times: user 13.8 s, sys: 3.7 s, total: 17.5 s
Wall time: 17.5 s


### Use Hyperopt for hyperparameter tuning

In [7]:
## Hyper Parameter Tuning for LGBM with HypterOPT
from hyperopt import hp, tpe, Trials, STATUS_OK
from hyperopt.fmin import fmin
from hyperopt.pyll.stochastic import sample
import lightgbm as lgb
import gc

#GLOBAL HYPEROPT PARAMETERS
NUM_EVALS = 1000 #number of hyperopt evaluation rounds
N_FOLDS = 5 #number of cross-validation folds on data in each evaluation round

#LIGHTGBM PARAMETERS
LGBM_MAX_LEAVES = 2**11 # maximum number of leaves per tree for LightGBM
LGBM_MAX_DEPTH = 25 # maximum tree depth for LightGBM
EVAL_METRIC_LGBM_REG = 'rmse' # LightGBM regression metric. Note that 'rmse' is more commonly used .. was 'mae'

def hyperopt_lgbm(data_train, labels_train, data_eval, labels_eval, num_evals=NUM_EVALS, diagnostic=False):
    
    print(f"'Running {num_evals} round(s) of LightGBM hyperparameter optimisation:")
    
    integer_params = ['max_depth', 'num_leaves', 'min_data_in_leaf', 'n_estimators']
    
    def objective(params):
        print(params)
        
        #cast integer params from float to int
        for param in integer_params:
            params[param] = int(params[param])

        num_round = int(params['n_estimators'])

        # Use train and predict
        # In case want to train and predict
        model = lgb.train(params, dtrain, num_round, valid_sets=dvalid, early_stopping_rounds=100, verbose_eval=False)
        y_pred = model.predict(data_eval)

        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        print(f"RMSE: {rmse}")
            
        return {'loss': rmse, 'status': STATUS_OK}
    
    dtrain = lgb.Dataset(data_train, label=labels_train)
    dvalid = lgb.Dataset(data_eval, label=labels_eval)
    watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
                
    #integer and string parameters, used with hp.choice()
    objective_list_reg = ['huber', 'fair']
    objective_list = objective_list_reg

    space = {'n_estimators': hp.quniform('n_estimators', 100, 250, 1),
             'boosting' : 'gbdt',
             'num_leaves' : hp.quniform('num_leaves', 2, LGBM_MAX_LEAVES, 1),
             'max_depth': hp.quniform('max_depth', 2, LGBM_MAX_DEPTH, 1),
             'min_data_in_leaf': hp.quniform('min_data_in_leaf', 1, 256, 1),
             'min_gain_to_split' : hp.quniform('min_gain_to_split', 0.1, 5, 0.01),
             'lambda_l1' : hp.uniform('lambda_l1', 0, 5),
             'lambda_l2' : hp.uniform('lambda_l2', 0, 5),
             'learning_rate' : hp.loguniform('learning_rate', np.log(0.005), np.log(0.2)),
             'metric' : 'RMSE',
             'objective' : 'regression',
             'feature_fraction' : hp.quniform('feature_fraction', 0.5, 1, 0.01),
             'bagging_fraction' : hp.quniform('bagging_fraction', 0.5, 1, 0.01),
             'verbose': -1,  # shut down logger output
             'feature_pre_filter' : False}

    # Define best trials object
    trials = Trials()
    best = fmin(fn=objective,
                space=space,
                algo=tpe.suggest,
                max_evals=num_evals, 
                trials=trials)

    # Print out best metrics and methods
    # best['objective'] = objective_list[best['objective']]
    
    #cast floats of integer params to int
    for param in integer_params:
        best[param] = int(best[param])
    
    print('{' + '\n'.join(f"{k}: {v}" for k, v in best.items()) + '}')
    
    if diagnostic:
        return (best, trials)
    else:
        return (best)

In [8]:
%%time
# lgbm_params, trials = hyperopt_lgbm(x_train, y_train, x_test, y_test, 1, diagnostic=True)

CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 9.06 µs


In [9]:
## IMPORTANT ##
## Using Hyperopt in Kaggle takes too much time or either returns an error 'Out of Memory'. I took the Hyperopt 
## training to a Google Colab notebook with 300 evals to find the best LGBM hyper parameters. Hence, 
## there is also only 1 eval conducted in the cell above just for demonstration purposes
## See below the best LGBM parameters from the training in Google Colab

best_lgbm_params = {'bagging_fraction': 0.99,
                    'boosting': 'gbdt',
                    'feature_fraction': 0.81,
                    'feature_pre_filter': False,
                    'lambda_l1': 4.293930802891639,
                    'lambda_l2': 1.1593070193619666,
                    'learning_rate': 0.06464035186294781,
                    'max_depth': 21,
                    'metric': 'RMSE',
                    'min_data_in_leaf': 151,
                    'min_gain_to_split': 0.58,
                    'n_estimators': 250,
                    'num_leaves': 1183,
                    'objective': 'regression',
                    'verbose': 1}

In [10]:
%%time
## Model with LGBM
import lightgbm as lgb

lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_test, y_test)

model = lgb.train(best_lgbm_params, lgb_train, valid_sets=lgb_eval, early_stopping_rounds=100)

y_expect  = y_test
y_pred = model.predict(x_test)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 3.803552 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 65280
[LightGBM] [Info] Number of data points in the train set: 2198987, number of used features: 256
[LightGBM] [Info] Start training from score -0.021636
[1]	valid_0's rmse: 0.920059
Training until validation scores don't improve for 100 rounds
[2]	valid_0's rmse: 0.919207
[3]	valid_0's rmse: 0.918451
[4]	valid_0's rmse: 0.917672
[5]	valid_0's rmse: 0.916922
[6]	valid_0's rmse: 0.916303
[7]	valid_0's rmse: 0.915752
[8]	valid_0's rmse: 0.915211
[9]	valid_0's rmse: 0.914714
[10]	valid_0's rmse: 0.914305
[11]	valid_0's rmse: 0.913847
[12]	valid_0's rmse: 0.913453
[13]	valid_0's rmse: 0.913078
[14]	valid_0's rmse: 0.912687
[15]	valid_0's rmse: 0.912338
[16]	valid_0's rmse: 0.911995
[17]	valid_0's rmse: 0.91167
[18]	valid_0's rmse: 0.911392
[19]	valid_0's rmse: 0.911105
[20]	valid_0's rmse: 0.91082

In [11]:
#RMSE
from sklearn.metrics import mean_squared_error

rmse = np.sqrt(mean_squared_error(y_expect, y_pred))
print(f"RMSE: {rmse}")

RMSE: 0.8903406784306958


## Submit

In [12]:
import ubiquant
env = ubiquant.make_env()  
iter_test = env.iter_test()

for (test_df, sample_prediction_df) in iter_test:
    
    test_df_feat = test_df.loc[:, list(x_train.columns)]
#     print(test_df_feat)
    lgb_pred = model.predict(test_df_feat)
    
    sample_prediction_df['target'] = lgb_pred
    print(sample_prediction_df)
    env.predict(sample_prediction_df) 
    display(sample_prediction_df)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
   row_id    target
0  1220_1 -0.157919
1  1220_2 -0.129462


,row_id,target
0,1220_1,-0.157919
1,1220_2,-0.129462


   row_id    target
0  1221_0 -0.129720
1  1221_1 -0.035262
2  1221_2 -0.246631


,row_id,target
0,1221_0,-0.129720
1,1221_1,-0.035262
2,1221_2,-0.246631


   row_id    target
0  1222_0 -0.053578
1  1222_1 -0.134988
2  1222_2 -0.071404


,row_id,target
0,1222_0,-0.053578
1,1222_1,-0.134988
2,1222_2,-0.071404


   row_id    target
0  1223_0 -0.231389


,row_id,target
0,1223_0,-0.231389
